In [2]:
import cv2
import os, glob
import numpy as np
import json
import shutil
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
 
image_path = "data/temp" # 원본 이미지의 경로
point_path = "data/json" # label의 경로
save = "data/masks" # mask를 저장할 경로
 
"""
폴더를 바꿔도 크게 상관은 없으나 파일 이름은 아래 형식과 같아야 합니다.
00001은 예시입니다.
원본 이미지 : 00001.jpg
mask 이미지: 00001_mask.jpg
"""
for p in os.listdir(point_path):
    masks = []
    with open(os.path.join(point_path, p), 'r') as f:
        data = json.load(f) # json 파일 읽어오기
        img_name = p.split(".")[0]+".jpg" # 원본 이미지의 파일 이름
        mask_name = p.split(".")[0]+"_mask.jpg" # mask의 파일 이름

        if not os.path.isfile(os.path.join(save, mask_name)):
            img = cv2.imread(os.path.join(image_path, img_name))
            cv2.imwrite(os.path.join("data/imgs", img_name), img)
            
            for d in data["shapes"]:
                data = d["points"] # 도형 하나씩 불러오기
                pts = np.array(data)
                pts = pts.astype(np.int32)
    
                mask = np.zeros(img.shape[:2], np.uint8) # 검은색 배경 
                cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA) # 사진위에 영역을 그리기
                dst = cv2.bitwise_and(img, img, mask=mask) # bitwise 연산(and)
    
                bg = np.zeros_like(img, np.uint8)*255
                res = bg + dst
                _, res = cv2.threshold(res, 1, 255, cv2.THRESH_BINARY)
                
                masks.append(res)
                
                if len(masks) > 1: masks[0] = cv2.bitwise_xor(masks[0], masks[-1])

                #     plt.subplot(1, 2, 1)
                #     plt.imshow(masks[0])
                #     plt.subplot(1, 2, 2)
                # plt.imshow(res)
                # plt.show()
            
            # mask[0]은 경추 전체를 덮는 도형 -> 나머지 도형들을 하나씩 덮어쓰는 형태

            _, masks[0] = cv2.threshold(masks[0], 0, 255, cv2.THRESH_BINARY)
            cv2.imwrite(os.path.join(save, mask_name), masks[0])